In [30]:
%%bash

pip install openai

In [60]:
from openai import OpenAI
import json
import pandas as pd
import yaml
import tqdm

In [33]:
data = pd.read_csv("shlokas.csv")
bg_df = data.drop(columns=["S.No.", "Sanskrit Anuvad", "Hindi Anuvad"])


bg_chapter_3_df = bg_df[bg_df["Chapter"] == 'Chapter 3']
bg_chapter_3_df = bg_chapter_3_df.drop(columns=["Chapter"])
bg_chapter_3_df.head()

,Title,Verse,Enlgish Translation
118,Karma Yoga,Verse 3.1,"Arjuna asked the Good Lord : O Lord KRISHNA, i..."
119,Karma Yoga,Verse 3.2,"I am confused dear Lord, by the advice you hav..."
120,Karma Yoga,Verse 3.3,"The Blessed Lord replied : O Arjuna, always re..."
121,Karma Yoga,Verse 3.4,"O Arjuna, it is not only important to understa..."
122,Karma Yoga,Verse 3.5,Lord Krishna continued:There is nobody who is ...


In [34]:
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=""
)
MODEL = "meta/llama-3.1-405b-instruct"

In [38]:
num_verses= len(bg_chapter_3_df)

TOPIC_GENERATION_PROMPT_TEMPLATE = """\
I want to create a synthetic dataset of specific and practical question answers on human daily problems. Based on this context, group {num_verses} verses in the context into groups as follows:
1. The context is made up of shlokas where arjuna asks a question followed by which krishna provides answers - call this group A
2. Sometimes krishna might explain things not asked by arjuna just to expound and give more knowledge - call this group K
3. For each group give a brief title describing what it talks about
4. Each group can either be of type group A or group K

Now group the following context into groups 1, 2, 3, ... so on in the format below each separated by a newline, There must be no other text than the following list:
```yaml
  response:
      - title: Title 1
        group: A
        verses:
          - Verse 1
          - Verse 2
          ...
      - title: Title 2
        group: K
        verses:
          - Verse 1
          - Verse 2
          ...
      ...
```

context:
```
{context}
```


"""

def generate_subtopics(client, num_verses, context):
    prompt = TOPIC_GENERATION_PROMPT_TEMPLATE.format(num_verses=num_verses, context=context)
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0.2,
        top_p=0.7,
    )
    return response

responses = generate_subtopics(client, num_verses=num_verses, context="\n\n".join(bg_chapter_3_df[["Verse","Enlgish Translation"]].apply(lambda x:  x[0] + " : " + x[1], axis=1).values.tolist()[:num_verses]))
print(responses.choices[0].message.content)

<ipython-input-38-521fe817e395>:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  responses = generate_subtopics(client, num_verses=num_verses, context="\n\n".join(bg_chapter_3_df[["Verse","Enlgish Translation"]].apply(lambda x:  x[0] + " : " + x[1], axis=1).values.tolist()[:num_verses]))


Here is the grouped context in the requested format:


  response:
      - title: Arjuna's Confusion and Krishna's Clarification
        group: A
        verses: 
          - Verse 3.1
          - Verse 3.2
          - Verse 3.3
          - Verse 3.4
      - title: The Importance of Action and Self-Control
        group: K
        verses: 
          - Verse 3.5
          - Verse 3.6
          - Verse 3.7
          - Verse 3.8
      - title: The Spirit of Sacrifice and Detachment
        group: K
        verses: 
          - Verse 3.9
          - Verse 3.10
          - Verse 3.11
          - Verse 3.12
          - Verse 3.13
          - Verse 3.14
          - Verse 3.15
      - title: The Consequences of Not Following the Proper Course
        group: K
        verses: 
          - Verse 3.16
          - Verse 3.17
          - Verse 3.18
      - title: Performing Actions Without Attachment
        group: A
        verses: 
          - Verse 3.19
          - Verse 3.20
      - title: The 

In [42]:
grouped_shlokas = yaml.safe_load(responses.choices[0].message.content)['Here is the grouped context in the requested format']['response']
grouped_shlokas

[{'title': "Arjuna's Confusion and Krishna's Clarification",
  'group': 'A',
  'verses': ['Verse 3.1', 'Verse 3.2', 'Verse 3.3', 'Verse 3.4']},
 {'title': 'The Importance of Action and Self-Control',
  'group': 'K',
  'verses': ['Verse 3.5', 'Verse 3.6', 'Verse 3.7', 'Verse 3.8']},
 {'title': 'The Spirit of Sacrifice and Detachment',
  'group': 'K',
  'verses': ['Verse 3.9',
   'Verse 3.10',
   'Verse 3.11',
   'Verse 3.12',
   'Verse 3.13',
   'Verse 3.14',
   'Verse 3.15']},
 {'title': 'The Consequences of Not Following the Proper Course',
  'group': 'K',
  'verses': ['Verse 3.16', 'Verse 3.17', 'Verse 3.18']},
 {'title': 'Performing Actions Without Attachment',
  'group': 'A',
  'verses': ['Verse 3.19', 'Verse 3.20']},
 {'title': 'The Importance of Setting a Good Example',
  'group': 'K',
  'verses': ['Verse 3.21', 'Verse 3.22', 'Verse 3.23', 'Verse 3.24']},
 {'title': 'Performing Actions Without Attachment (Continued)',
  'group': 'K',
  'verses': ['Verse 3.25', 'Verse 3.26']},
 {'

```json
[{'title': "Arjuna's Confusion and Krishna's Clarification",
  'group': 'A',
  'verses': ['Verse 3.1', 'Verse 3.2', 'Verse 3.3', 'Verse 3.4']},
 {'title': 'The Importance of Action and Self-Control',
  'group': 'K',
  'verses': ['Verse 3.5', 'Verse 3.6', 'Verse 3.7', 'Verse 3.8']},
 {'title': 'The Spirit of Sacrifice and Detachment',
  'group': 'K',
  'verses': ['Verse 3.9',
   'Verse 3.10',
   'Verse 3.11',
   'Verse 3.12',
   'Verse 3.13',
   'Verse 3.14',
   'Verse 3.15']},
 {'title': 'The Consequences of Not Following the Proper Course',
  'group': 'K',
  'verses': ['Verse 3.16', 'Verse 3.17', 'Verse 3.18']},
 {'title': 'Performing Actions Without Attachment',
  'group': 'A',
  'verses': ['Verse 3.19', 'Verse 3.20']},
 {'title': 'The Importance of Setting a Good Example',
  'group': 'K',
  'verses': ['Verse 3.21', 'Verse 3.22', 'Verse 3.23', 'Verse 3.24']},
 {'title': 'Performing Actions Without Attachment (Continued)',
  'group': 'K',
  'verses': ['Verse 3.25', 'Verse 3.26']},
 {'title': 'Understanding the Modes of Nature',
  'group': 'K',
  'verses': ['Verse 3.27', 'Verse 3.28', 'Verse 3.29']},
 {'title': 'Surrendering Actions to Krishna',
  'group': 'A',
  'verses': ['Verse 3.30', 'Verse 3.31', 'Verse 3.32']},
 {'title': 'The Futility of Restraint',
  'group': 'K',
  'verses': ['Verse 3.33']},
 {'title': 'Overcoming Attachment and Desire',
  'group': 'K',
  'verses': ['Verse 3.34', 'Verse 3.35']},
 {'title': 'The Cause of Sin',
  'group': 'A',
  'verses': ['Verse 3.36',
   'Verse 3.37',
   'Verse 3.38',
   'Verse 3.39',
   'Verse 3.40',
   'Verse 3.41',
   'Verse 3.42',
   'Verse 3.43']}]
```

```
{'Dilemma of Action': ['Verse 3.1', 'Verse 3.2'],
 'Two Paths of Action': ['Verse 3.3', 'Verse 3.4', 'Verse 3.5'],
 'Importance of Self-Control': ['Verse 3.6', 'Verse 3.7'],
 'Action vs Inaction': ['Verse 3.8', 'Verse 3.9'],
 'Sacrifice and Growth': ['Verse 3.10', 'Verse 3.11', 'Verse 3.12'],
 'Importance of Sacrifice': ['Verse 3.13', 'Verse 3.14', 'Verse 3.15'],
 'Consequences of Not Following the Proper Course': ['Verse 3.16'],
 'State of Perfection': ['Verse 3.17', 'Verse 3.18', 'Verse 3.19'],
 'Performing Duties Without Attachment': ['Verse 3.20', 'Verse 3.21'],
 'Setting an Example': ['Verse 3.22', 'Verse 3.23', 'Verse 3.24'],
 'Preserving World Order': ['Verse 3.25', 'Verse 3.26'],
 'Three Modes of Nature': ['Verse 3.27', 'Verse 3.28', 'Verse 3.29'],
 'Surrendering to the Divine': ['Verse 3.30', 'Verse 3.31', 'Verse 3.32'],
 "Nature's Influence on Actions": ['Verse 3.33'],
 'Overcoming Attachment': ['Verse 3.34', 'Verse 3.35'],
 'Motivation for Sin': ['Verse 3.36', 'Verse 3.37'],
 'Desire as an Enemy': ['Verse 3.38', 'Verse 3.39', 'Verse 3.40'],
 'Controlling the Senses': ['Verse 3.41', 'Verse 3.42', 'Verse 3.43']}
 ```

In [43]:
verse_map = bg_chapter_3_df[["Verse","Enlgish Translation"]].set_index("Verse")
verse_map.head()

,Enlgish Translation
Verse,
Verse 3.1,"Arjuna asked the Good Lord : O Lord KRISHNA, i..."
Verse 3.2,"I am confused dear Lord, by the advice you hav..."
Verse 3.3,"The Blessed Lord replied : O Arjuna, always re..."
Verse 3.4,"O Arjuna, it is not only important to understa..."
Verse 3.5,Lord Krishna continued:There is nobody who is ...


In [44]:
n_questions = 50

INSTRUCTION_PROMPT_TEMPLATE = """\
The objective is to create a dataset of specific and practical questions on daily issues faced by humans.
These questions should be comprehensive of different walks of life - personal, professional, family, education, health, friends, (add more of your own here).

Given a bunch of shlokas from bhagavad gita, generate {n_questions} possible concise questions that could be given to an AI assitant about that topic from various walks of life.
Write some of these questions as if given by someone with limited knowledge of bhagavad gita, like a beginner. Your response should be in a list format.

Shlokas:
```
{shlokas}
```

The list must be without numbers. The questions should be separated by a newline character. There must be no other text than the list.
"""

def generate_questions(client, context_dict, n_questions):
    print(f"Generating Instructions for {context_dict}.")
    prompt = INSTRUCTION_PROMPT_TEMPLATE.format(n_questions=n_questions, shlokas=context_dict['shlokas'])
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0.2,
        top_p=0.7,
    )
    return response.choices[0].message.content




In [49]:
with open("bhagavad_gita_chapter_3_q.jsonl", "w") as f:
  for group in grouped_shlokas:
    context_dict = {
        "title": group['title'],
        "group": group['group'],
        "shlokas": "\n\n".join([verse_map.loc[verse, "Enlgish Translation"] for verse in group['verses']])
    }
    questions = generate_questions(client, context_dict, n_questions)
    for question in questions.split("\n"):
      if question == "":
        continue
      f.write(json.dumps({
        "topic": group['title'],
        "verses": group['verses'],
        "shlokas": context_dict['shlokas'],
        "question": question
      }))
      f.write('\n')




Generating Instructions for {'title': "Arjuna's Confusion and Krishna's Clarification", 'group': 'A', 'shlokas': 'Arjuna asked the Good Lord\xa0: O Lord KRISHNA, if you claim that Gyan or Knowledge is better than one’s Karma or responsibility, the why do you ask me to fight this battle?\n\nI am confused dear Lord, by the advice you have given to me. You have told me to take two opposite and different courses of action at once. Please, O KRISHNA, tell me of just one wise solution and course of action that would lead out of this problem safely.\n\nThe Blessed Lord replied\xa0: O Arjuna, always remember in life that there are only two definite paths of action as I have described before. One of these is known as Sankhyayoga or the path of knowledge, and the other is Karmayoga, or the path of performing duty and action without expecting any result.\n\nO Arjuna, it is not only important to understand the meanings of Sankhyayoga and Karmayoga, but it is also important to understand the aims o

In [52]:
with open(f'bhagavad_gita_chapter_3_q.jsonl', 'r') as f:
    question_records = [json.loads(line) for line in f]

question_records[:5]

[{'topic': "Arjuna's Confusion and Krishna's Clarification",
  'verses': ['Verse 3.1', 'Verse 3.2', 'Verse 3.3', 'Verse 3.4'],
  'shlokas': 'Arjuna asked the Good Lord\xa0: O Lord KRISHNA, if you claim that Gyan or Knowledge is better than one’s Karma or responsibility, the why do you ask me to fight this battle?\n\nI am confused dear Lord, by the advice you have given to me. You have told me to take two opposite and different courses of action at once. Please, O KRISHNA, tell me of just one wise solution and course of action that would lead out of this problem safely.\n\nThe Blessed Lord replied\xa0: O Arjuna, always remember in life that there are only two definite paths of action as I have described before. One of these is known as Sankhyayoga or the path of knowledge, and the other is Karmayoga, or the path of performing duty and action without expecting any result.\n\nO Arjuna, it is not only important to understand the meanings of Sankhyayoga and Karmayoga, but it is also importa

In [53]:
RESPONSE_PROMPT_TEMPLATE = """\
You are a therapist who solves humans' daily problems and miseries. Your solution should be based on bhagavad gita, WITHIN the context provided.

Follow these rules before responding:
1. ONLY USE THE CONTEXT PROVIDED IN THE PROMPT.
2. RETURN EMPTY STRING '' IF YOU CANNOT ANSWER BASED ON THE CONTEXT.
3. BE VERY PRACTICAL, AND GIVE EXAMPLES WHEREVER NECESSARY.
4. KEEP THE RESPONSE ON-TOPIC, INFORMATIVE, PRACTICAL, AND CONCISE.

context:
```
{context}
```

The question is: {question}
"""
def generate_responses(client, context, question):
    prompt = RESPONSE_PROMPT_TEMPLATE.format(question=question, context=context)
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    return response.choices[0].message.content

In [54]:
import random

random.sample(question_records, 5)

[{'topic': 'Understanding the Modes of Nature',
  'verses': ['Verse 3.27', 'Verse 3.28', 'Verse 3.29'],
  'shlokas': 'The Blessed Lord Krishna spoke: O Arjuna, all actions that are performed by beings, are done so by three modes of Prakrithi or three aspects of nature. However, ignorant people claim themselves as the performer of their actions.\n\nHe who understands fully, O Arjuna, the objects of perception (senses, mind, etc.), divisions of Prakrithi (nature), and selfless action, has the ability to understand the relationship of senses and sensual objects and never gets attached to any beings or objects.\n\nThe Lord spoke: Those who are deceived by the Gunas of nature or Prakrithi and are deceived by karma, develop an attachment to the Gunas and to their actions.A wise person would not disturb, or have anything to do with these ignorant beings, O Arjuna.',
  'question': 'As a parent, how can I teach my children about the three modes of Prakrithi and their influence on our actions?'}

In [61]:
with open("bhagavad_gita_chapter_3_qa.jsonl", "w") as f:
  for question_record in tqdm.tqdm(question_records):
      context = question_record['shlokas']
      question = question_record['question']
      response = generate_responses(client, context, question)
      question_record['response'] = response
      # print(f"Question: {question}")
      # print(f"Response: {response}")
      # print(f"Context: {context}")
      # print(f"----------------------------------------")
      f.write(json.dumps(question_record))
      f.write('\n')

100%|██████████| 571/571 [2:16:38<00:00, 14.36s/it]


In [62]:
from google.colab import drive
drive.mount('/content/drive')

!cp bhagavad_gita_chapter_3_qa.jsonl /content/drive/MyDrive/
!cp bhagavad_gita_chapter_3_q.jsonl /content/drive/MyDrive/
!cp /content/bhagavad_gita_chapter_3_questions.jsonl /content/drive/MyDrive/
!cp shlokas.csv /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
context

'Arjuna asked the Good Lord\xa0: O Lord KRISHNA, if you claim that Gyan or Knowledge is better than one’s Karma or responsibility, the why do you ask me to fight this battle?\n\nI am confused dear Lord, by the advice you have given to me. You have told me to take two opposite and different courses of action at once. Please, O KRISHNA, tell me of just one wise solution and course of action that would lead out of this problem safely.'

In [ ]:


instruction_response_list = response_generator(client, instruction_list_formatted)
instruction_response_pair_list = []
for instruction, response in zip(instruction_list_formatted, instruction_response_list):
    instruction_response_pair_list.append(
        {
            "instruction": instruction,
            "responses": response,
        }
    )